In [13]:
from __future__ import division

from math import floor
import numpy as np

In [132]:
# Parameters
module = 0.7
max_external_diam = 32 #mm
target_reduction = 5 
min_shell_tickness = 1 #mm
min_z_sun = 15 #nb de dent

In [133]:
class find_planetary_gearbox_parameters(object):
    def __init__(self, target_reduction, module, max_external_diam, min_shell_tickness, min_z_sun=5):
        max_diam_ref_ring = max_external_diam - 2*min_shell_tickness - 2*module
        
        self.module = module
        self.target_reduction = target_reduction
        self.min_z_sun = min_z_sun
        
        #init research
        
        self.z_ring = floor(max_diam_ref_ring / self.module)
        self.z_sun = self.z_ring / (target_reduction-1)
        self.z_planet = self.compute_z_planet(self.z_sun, self.z_ring, self.module)
        
        if self.is_gearbox_valid(self.z_sun, self.z_ring, self.z_planet):
            print 'good:', self.z_sun, self.z_ring, self.z_planet
            return
        else:
            self.result =  self.find_correct_gear()
            #self.extract_best_results(self.result, 3)
            
    def find_correct_gear(self):
        z_ring_test = np.arange(self.z_ring-10, self.z_ring)
        z_sun_test = np.arange(max(self.min_z_sun, self.z_sun-10), self.z_sun+10)
        result = []
        
        for z_ring in z_ring_test:
            for z_sun in z_sun_test:
                z_planet = self.compute_z_planet(z_sun, z_ring, self.module)
                
                if self.is_gearbox_valid(z_sun, z_ring, z_planet):
                    r = self.compute_reduction(z_sun, z_ring)
                    result.append([z_sun, z_ring, z_planet, r, abs(self.target_reduction-r)])
        
        self.extract_best_results(result, 5)
        
        return result

    def extract_best_results(self, result, n):
        result = np.array(result)
        sort = result[result[:,4].argsort()] # sort on reduction error
        
        print n, 'best solutions:'
        for i in range(n):
            print 'z_sun =',int(sort[i,0]), ', z_ring =',int(sort[i,1]), ', z_planet =',int(sort[i,2]), ', reduction =',sort[i,3],'\n'
        
            
    def compute_z_planet(self, z_sun, z_ring, module):
        d_sun = module * z_sun
        d_ring = module * z_ring
    
        d_planet = (d_ring - d_sun) / 2
    
        return d_planet / module
    
    def is_gearbox_valid(self, z_sun, z_ring, z_planet):
        if z_sun.is_integer() and z_planet.is_integer() and z_ring.is_integer():
            return True
        else:
            return False
        
    def compute_reduction(self, z_sun, z_ring):
        return (z_sun+z_ring)/z_sun

In [134]:
r = find_planetary_gearbox_parameters(target_reduction, module, max_external_diam, min_shell_tickness, min_z_sun)

5 best solutions:
z_sun = 15 , z_ring = 37 , z_planet = 11 , reduction = 3.46666666667 

z_sun = 16 , z_ring = 38 , z_planet = 11 , reduction = 3.375 

z_sun = 15 , z_ring = 35 , z_planet = 10 , reduction = 3.33333333333 

z_sun = 17 , z_ring = 39 , z_planet = 11 , reduction = 3.29411764706 

z_sun = 16 , z_ring = 36 , z_planet = 10 , reduction = 3.25 

